# Solar Energy on Building Envelopes (SEBE)

For the solar irradiance modelling, (Lindberg et al., 2015) SEBE is used. SEBE is incorporated in Urban Multi-scale Environmental Predictor (UMEP) (Sun et al., 2020) as a plugin for QGIS. It uses DSM to calculate pixel-wise potential solar irradiance on building rooftops, walls, and ground surfaces.  SEBE uses a shadow casting algorithm to generate shadows, which helps determine the shading losses on rooftop segments caused by neighbouring buildings and vegetation. 

# Prerequisite script

1. Attribute Table.ipynb
2. Panel Distribution Algorithm.ipynb

In [7]:
from osgeo import gdal, osr
import pandas as pd
import numpy as np
from sunmapcreator_2015a import sunmapcreator_2015a
from Solweig_2015a_metdata_noload import Solweig_2015a_metdata_noload
from sebeworker import Worker
import geopandas as gpd
from pyproj import Transformer
from pvlib import solarposition
import pvlib
import math
import time
from PIL import Image
import statistics

In [8]:
    """
    
    Input parameters:
    -----------------
    
    The primary inputs for this script are raster and vector datasets.
    
    gdal_dsm          : This DSM is larger than the study area, in order to eliminate the boundary effects and
                        to account for proper object shading.
                        DSM consisting of pixels with elevation heights above ground.
    slope             : Slope of a segment is calculated from DSM by using Geospatial Data Abstraction Library 
                        (GDAL) Spatial Analyst toolbox in QGIS. (GDAL (GDAL/OGR contributors, 2021))) 
                        After generating the slope raster, slopes are reclassified. In this reclassification, 
                        slopes higher than 60° are considered walls and hence eliminated by allocating slopes 
                        ranging in between 60° and 90° to -1 and slopes < 10° are considered flat and valued to 0°
    aspect            : The aspect tool of the GDAL Spatial Analyst Toolbox in QGIS was used to obtain aspect values from DSM
    
    umep_wh           : "Urban Geometry: Wall Height used to identify wall pixels and their height from ground and DSM 
                        by using a filter as presented by Lindberg et al. (2015a)"
    umep_wa           : "Urban Geometry: Wall Aspect is estimated using a specific linear filter as presented 
                        by Goodwin et al. (1999) and further developed by Lindberg et al. (2015b) to obtain the wall aspect. 
                        Wall aspect is given in degrees where a north facing wall pixel has a value of zero."
    shapefile/                   
    direct_PVP/       : Output shapefile from "Panel distribution algorithm.ipynb" that contains information of panels in urban area
    diffuse_PVP/
    reflected_PVP
      
    segments          : Output shapefile from "Attribute table.ipynb" contains information of rooftop segments.
    vegdsm            : A DSM consisting of pixels with vegetation heights above ground.
    
    Further Information:
    --------------------
https://umep-docs.readthedocs.io/en/latest/processor/Solar%20Radiation%20Solar%20Energy%20on%20Building%20Envelopes%20(SEBE).html
    
    
    Note: Input files and parameters can be found under the folder "Datasets" in Github
    
    Reference:
    ----------
    
                        (Lindberg, Fredrik & Grimmond, Christine & Gabey, Andrew & Huang, Bei & Kent, Christoph & Sun, 
                        Ting & Theeuwes, Natalie & Järvi, Leena & Ward, Helen & Capel-Timms, Isabella & Chang, 
                        Yuanyong & Jonsson, Per & Krave, Niklas & Liu, Dongwei & Meyer, D. & Olofson, Frans & Tan, 
                        Jianguo & Wästberg, Dag & Xue, Lingbo & Zhang, Zhe. (2018). 
                        Urban Multi-scale Environmental Predictor (UMEP): An integrated tool for city-based climate services. 
                        Environmental Modelling & Software. 99. 70-87. 10.1016/j.envsoft.2017.09.020.)
                        
                        (Freier, Daria & Muhammad-Sukki, Firdaus & Abu-Bakar, Siti Hawa & Ramirez-Iniguez, Roberto & Munir, 
                        Abu & Mohd Yasin, Siti Hajar & Bani, Nurul & Mas'ud, Abdullahi & Ardila-Rey, Jorge & Karim, Md. (2018).
                        Annual prediction output of an RADTIRC-PV module. Energies. 11. 544. 10.3390/en11030544.) 
    
    """
    
gdal_dsm      = gdal.Open('dsm.tif')
slope         = gdal.Open('dsm_reclassify_slope.tif') # Flat slope raster
aspect        = gdal.Open('dsm_aspect.tif')
umep_wh       = gdal.Open('wheight.tif')
umep_wa       = gdal.Open('waspect.tif')
shapefile     = gpd.read_file('panels_grid.shp')
direct_PVP    = gpd.read_file('panels_grid.shp')
diffuse_PVP   = gpd.read_file('panels_grid.shp')
reflected_PVP = gpd.read_file('panels_grid.shp')
segments      = gpd.read_file('segments.shp')
vegdsm        = gdal.Open('vegetation.tif')


albedo       = 0.2     # System albedo for an urban area
d            = 1.5     # Panel row spacing [m]
l            = 1.6     # Length of PV panel in landscape mode [m]
w            = 1       # Width of PV panel in landscape mode [m]
β_o          = 53      # Optimized tilt angle for panels installed on flat rooftop segments [deg]
wire_losses  = 0.98    # Wiring losses
pv_eff       = 0.177   # Panel efficiency [%]
alpha        = -0.0039 # Temperature correction factor of the panel [1/K]
k_t          = 0.05    # Reduction factor, default value = 0.05 °C/(W/m2)
ref_temp     = 25      # Reference PV cell temperature, default value = 25 °C

# Input meteorological data specifically formatted to be used in UMEP
# metdata = np.loadtxt("Berlin_Weather.txt", skiprows=1, dtype=float).reshape((-1, 24))
metdata = np.loadtxt("june_RMD.txt", skiprows=1, dtype=float).reshape((-1, 24))
UTC     = 2       # Set UTC according to your location and the time.

In [9]:
# Reading the shapefiles into an array

dsm             = gdal_dsm.ReadAsArray().astype(np.float)
building_slope  = slope.ReadAsArray().astype(np.float)
building_aspect = aspect.ReadAsArray().astype(np.float)
wheight         = umep_wh.ReadAsArray().astype(np.float)
waspect         = umep_wa.ReadAsArray().astype(np.float)
vegdsm          = vegdsm.ReadAsArray().astype(np.float)

for i in shapefile.index:
    
    shapefile.loc[i, 'ymin'] = int(shapefile.loc[i, 'geometry'].bounds[1])

In [10]:
# Extracting Geodata from DSM

sizex        = dsm.shape[0]
sizey        = dsm.shape[1]
width        = gdal_dsm.RasterXSize
height       = gdal_dsm.RasterYSize
geotransform = gdal_dsm.GetGeoTransform()
minx         = geotransform[0]
miny         = geotransform[3] + width * geotransform[4] + height * geotransform[5]
scale        = 1 / geotransform[1]
voxelheight  = geotransform[1]
panel_area   = l * w

In [11]:
for i in range(metdata.shape[0]):

    if metdata[[i], 14] > 0:
        
        onlyglobal   = 0
        usevegdem    = 1
        calc_month   = False
        psi          = 0.03
        TrunkHeight  = 25
        trunkratio   = TrunkHeight / 100.0
        vegdsm2      = vegdsm * trunkratio
        output       = {'energymonth': 0, 'energyyear': 1, 'suitmap': 0}  
        metdata1      = metdata[[i], :]
        hour         = int(metdata1[0,2])
        old_cs       = osr.SpatialReference()
        dsm_ref      = gdal_dsm.GetProjection()
        old_cs.ImportFromWkt(dsm_ref)
        new_cs    = osr.SpatialReference()
        
        wgs84_wkt = """
        GEOGCS["WGS 84",
            DATUM["WGS_1984",
                SPHEROID["WGS 84",6378137,298.257223563,
                    AUTHORITY["EPSG","7030"]],
                AUTHORITY["EPSG","6326"]],
            PRIMEM["Greenwich",0,
                AUTHORITY["EPSG","8901"]],
            UNIT["degree",0.01745329251994328,
                AUTHORITY["EPSG","9122"]],
            AUTHORITY["EPSG","4326"]]"""
        new_cs.ImportFromWkt(wgs84_wkt)
        transform = osr.CoordinateTransformation(old_cs, new_cs)
        lonlat    = transform.TransformPoint(minx, miny)
        gdalver   = float(gdal.__version__[0])
        
        
        if gdalver == 3.:
            
            lon = lonlat[1] #changed to gdal 3
            lat = lonlat[0] #changed to gdal 3
            
        else:
            
            lon = lonlat[0] #changed to gdal 2
            lat = lonlat[1] #changed to gdal 2
            
        # Prepare metdata
        alt = np.median(dsm)
        
        if alt < 0:
            
            alt = 3
        
        location = {'longitude': lon, 'latitude': lat, 'altitude': alt}
        
        YYYY, altitude, azimuth, zen, jday, leafon, dectime, altmax, timestamp = Solweig_2015a_metdata_noload(metdata1,location, UTC)
        
        radmatI, radmatD, radmatR = sunmapcreator_2015a(metdata1, altitude, azimuth, 
                                                        onlyglobal, output, jday, albedo, location, zen)
        
        total, direct, diffuse, reflected = Worker(dsm, scale, building_slope,building_aspect, voxelheight, sizey, sizex, 
                        vegdsm, vegdsm2, wheight, waspect, albedo, psi, radmatI, radmatD, radmatR, usevegdem, calc_month)
        
                
        print(f'{timestamp}')
        
        for j in shapefile.index:
                
                poly2         = shapefile.loc[j, 'geometry']
                bounds        = poly2.bounds
                geo_transform = gdal_dsm.GetGeoTransform ()
                
                pixel_y = int((((bounds[0] + bounds[2]) / 2) - geo_transform[0]) // geo_transform[1])
                pixel_x = int((((bounds[1] + bounds[3]) / 2) - geo_transform[3]) // (geo_transform[5]))
                
                Glob_hor = total[pixel_x, pixel_y]
                Diff_hor = diffuse[pixel_x, pixel_y]
                Dir_hor  = direct[pixel_x, pixel_y]
                ref      = reflected[pixel_x, pixel_y]
                    
                slope1  = shapefile.loc[j, 'Slope']
                aspect1 = shapefile.loc[j, 'Seg_Aspect']
                phi     = azimuth[0]
                gamma   = altitude[0]
                zenith  = zen[0] * 57.3 # 57.3 is an converter
                aoi     = pvlib.irradiance.aoi(slope1, aspect1, zenith, phi)
                
                # Calculate POA irradiance with self-shading of panels
                if gamma > 0 :
                    
                    lamda = math.cos(math.radians(phi - aspect1)) + (math.tan(math.radians(gamma)) / math.tan(math.radians(slope1)))
                    
                    if lamda <= 0 :
                        
                        f_s1 = 1
                        f_s2 = 1
                        
                    else:
                        
                        f_s1 = 0
                        x_l  = (math.cos(math.radians(slope1)) + d/l) * math.sin(math.radians(phi - aspect1)) / lamda
                        y_l  = 1 - ((math.cos(math.radians(slope1)) + d/l) * math.tan(math.radians(gamma)) / (lamda * math.sin(math.radians(slope1))))
                        
                        if y_l <=0:
                            
                            f_s2 = 0
                            
                        elif abs(x_l) > (w / l):
                            
                            f_s2 = 0
                            
                        else:
                            
                            f_s2 = (1 - (abs(x_l)/ (w / l))) * y_l
                    
                    f_sky1 = (1 + math.cos(math.radians(slope1))) / 2
                    f_sky2 = (1 + math.cos(math.radians(slope1)) + (d/l) - math.sqrt(((math.sin(math.radians(slope1)))**2) + (d/l)**2)) / 2
                    
                    D1 = f_sky1 * Diff_hor
                    D2 = f_sky2 * Diff_hor    
                    A1 = ref * (1 - f_sky1)
                    A2 = ref * (1 - f_sky2)     
                    B  = Dir_hor * (math.cos(math.radians(aoi)) / math.sin(math.radians(gamma)))
    
                    G_s1_B = (1 - f_s1) * B
                    G_s2_B = (1 - f_s2) * B
                    G_s1_D = D1
                    G_s2_D = D2
                    G_s1_R = A1
                    G_s2_R = A2

                    
                    if shapefile.loc[j, 'Slope'] == β_o:
                        
                        flat_segments = shapefile[(shapefile['segment_id'] == shapefile.loc[j, 'segment_id'])]
                        n_r           = pd.unique(flat_segments['ymin'])
                        
                        if shapefile.loc[j, 'ymin'] == n_r.min():
                            

                            direct_PVP.loc[j, str(hour)]    = G_s1_B * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s1_B) - ref_temp)) * panel_area * wire_losses
                            diffuse_PVP.loc[j, str(hour)]   = G_s1_D * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s1_D) - ref_temp)) * panel_area * wire_losses
                            reflected_PVP.loc[j, str(hour)] = G_s1_R * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s1_R) - ref_temp)) * panel_area * wire_losses
                        
                        else:

                            direct_PVP.loc[j, str(hour)]    = G_s2_B * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s2_B) - ref_temp)) * panel_area * wire_losses
                            diffuse_PVP.loc[j, str(hour)]   = G_s2_D * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s2_D) - ref_temp)) * panel_area * wire_losses
                            reflected_PVP.loc[j, str(hour)] = G_s2_R * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s2_R) - ref_temp)) * panel_area * wire_losses
                
                    else:

                        direct_PVP.loc[j, str(hour)]    = Dir_hor * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * Dir_hor) - ref_temp)) * panel_area * wire_losses
                        diffuse_PVP.loc[j, str(hour)]   = G_s1_D * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s1_D) - ref_temp)) * panel_area * wire_losses
                        reflected_PVP.loc[j, str(hour)] = G_s1_R * pv_eff * (1 + alpha * ((metdata1[:, 11] + k_t * G_s1_R) - ref_temp)) * panel_area * wire_losses
                        
                else:
                    
                        direct_PVP.loc[j, str(hour)]    = 0
                        diffuse_PVP.loc[j, str(hour)]   = 0
                        reflected_PVP.loc[j, str(hour)] = 0
                        
print("SEBE analysis is done")          

2019-06-16 05:00:00
2019-06-20 06:00:00
2019-06-09 07:00:00
2019-06-01 08:00:00
2019-06-24 09:00:00
2019-06-28 10:00:00
2019-06-18 11:00:00
2019-06-23 12:00:00
2019-06-05 13:00:00
2019-06-14 14:00:00
2019-06-17 15:00:00
2019-06-25 16:00:00
2019-06-12 17:00:00
2019-06-24 18:00:00
2019-06-27 19:00:00
2019-06-07 20:00:00
2019-06-24 21:00:00
SEBE analysis is done


In [12]:
direct_PVP.to_file("direct_PVP.shp")
diffuse_PVP.to_file("diffuse_PVP.shp")
reflected_PVP.to_file("reflected_PVP.shp")